# Exercise 1 - SAP TechEd 2025: DA261 Hands-On Workshop

The changelog of enhancements with the release version 2.25 can be found [in the documentation](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2025_2_QRC/en-US/change_log.html), and are summaried below  <br><br>

__Topics__<br>
    - ACDOCA Outlier / Universal Journal <br>
    - Service ticket classification, clustering, vector, text embedding, text chunking/tokenization, TA sentiment analyszs ... STI example, <br>
    - KGE / SparQL  <br><br>

VS Codes / BAS SQL Console and Python ....
    
<br>

## Section 1.0 - Connect to your SAP HANA Cloud instance

The latest Python ML client for SAP HANA package update is ready for installation and can be found at: https://pypi.org/project/hana-ml/

In [45]:
## Loading the Python Machine Learning client library for SAP HANA and get the version
import hana_ml
print(hana_ml.__version__)

2.25.25080800


In [ ]:
%run "./ex0/ex0_2_-check_setup.ipynb"

Connect to SAP HANA Cloud successful:  True  , with version:  4.00.000.00.1754989558 (fa/CE2025.14)
Current time on the SAP HANA Cloud server: 2025-08-14 09:08:08.870000


## Section 1.1 - Isolation Forest - ACDOCA Outlier

### 1.1 - Step 1: Create HANA dataframe on outlier data

#### Introduction to SAP HANA dataframes
- diff ways to create a dataframe (from pandas/spark, table, sql, view, multi-statement, ...
- some methods on dataframe
- collect method

In [47]:
acdoca_hdf = myconn.table("ACDOCA")
print(acdoca_hdf.select_statement)

SELECT * FROM "ACDOCA"


In [48]:
display(acdoca_hdf.collect())

,Company Code,G/L Account,Profit Center,Cost Center,Functional Area,Business Area,Segment,Debit/Credit,Accounting Document Type,Transaction Type,Financial Account Type,Amount (USD),Amount (Transaction)
0,CC01,550000,PC002,C101,FA01,BA02,S1,S,SC,TA02,P+L Statement,4158.39,-4158.39
1,CC01,780000,PC003,C102,FA02,BA03,S3,H,SC,TA03,Balance Sheet Asset,16137.40,-16137.40
2,CC01,510000,PC002,C102,FA01,BA03,S1,S,SD,TA03,P+L Statement,18333.69,-18333.69
3,CC01,470000,PC002,C102,FA01,BA02,S2,H,SC,TA01,Equity,1825.55,-1825.55
4,CC01,570000,PC001,C103,FA01,BA01,S1,H,SD,TA03,Balance Sheet Asset,19564.18,19564.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,CC01,460000,PC002,C101,FA02,BA02,S2,H,SA,TA01,Balance Sheet Liability,19772.27,19772.27
496,CC01,520000,PC001,C101,FA01,BA02,S2,S,SB,TA03,Balance Sheet Liability,11432.26,-11432.26
497,CC01,470000,PC003,C103,FA02,BA03,S3,H,SA,TA01,Equity,9014.75,-9014.75
498,CC01,790000,PC003,C102,FA02,BA02,S3,S,SD,TA03,Equity,12937.31,12937.31


In [49]:
acdoca_hdf=acdoca_hdf.add_id()

In [50]:
#display(df)
#display(acdoca_hdf.collect())
acdoca_hdf.dtypes()

[('ID', 'INT', 10, 10, 10, 0),
 ('Company Code', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('G/L Account', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Profit Center', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Cost Center', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Functional Area', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Business Area', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Segment', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Debit/Credit', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Accounting Document Type', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Transaction Type', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Financial Account Type', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('Amount (USD)', 'DOUBLE', 15, 15, 15, 0),
 ('Amount (Transaction)', 'DOUBLE', 15, 15, 15, 0)]

#### Filter data for outlier data

In [51]:
#Filter by rows / Where Clause
acdoca_hdf=acdoca_hdf.filter('"Company Code" = \'CC01\' AND "Profit Center"=\'PC002\'')
acdoca_hdf.select_statement
display(acdoca_hdf.collect())

,ID,Company Code,G/L Account,Profit Center,Cost Center,Functional Area,Business Area,Segment,Debit/Credit,Accounting Document Type,Transaction Type,Financial Account Type,Amount (USD),Amount (Transaction)
0,1,CC01,550000,PC002,C101,FA01,BA02,S1,S,SC,TA02,P+L Statement,4158.39,-4158.39
1,3,CC01,510000,PC002,C102,FA01,BA03,S1,S,SD,TA03,P+L Statement,18333.69,-18333.69
2,4,CC01,470000,PC002,C102,FA01,BA02,S2,H,SC,TA01,Equity,1825.55,-1825.55
3,6,CC01,410000,PC002,C101,FA02,BA01,S3,S,SC,TA01,Balance Sheet Liability,10073.36,-10073.36
4,13,CC01,640000,PC002,C103,FA01,BA01,S1,S,SB,TA03,Balance Sheet Asset,12119.38,12119.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,484,CC01,700000,PC002,C103,FA01,BA02,S3,S,SB,TA02,Balance Sheet Asset,12661.97,12661.97
163,485,CC01,530000,PC002,C101,FA02,BA01,S3,H,SA,TA02,Balance Sheet Asset,7931.05,-7931.05
164,489,CC01,480000,PC002,C102,FA02,BA02,S2,S,SC,TA02,Balance Sheet Asset,5533.86,5533.86
165,491,CC01,420000,PC002,C101,FA02,BA02,S1,H,SB,TA02,Equity,10298.93,10298.93


In [52]:
acdoca_hdf.select_statement

'SELECT * FROM (SELECT CAST(ROW_NUMBER() OVER() AS INTEGER) + 0 AS "ID", * FROM (SELECT * FROM "ACDOCA")) AS "DT_97" WHERE "Company Code" = \'CC01\' AND "Profit Center"=\'PC002\''

In [53]:
# Filter columns
acdoca_hdf2=acdoca_hdf.select('ID', 'Amount (USD)', 'Amount (Transaction)')
display(acdoca_hdf2.collect())

,ID,Amount (USD),Amount (Transaction)
0,1,4158.39,-4158.39
1,3,18333.69,-18333.69
2,4,1825.55,-1825.55
3,6,10073.36,-10073.36
4,13,12119.38,12119.38
...,...,...,...
162,484,12661.97,12661.97
163,485,7931.05,-7931.05
164,489,5533.86,5533.86
165,491,10298.93,10298.93


## Section 1.2 - Outlier analysis using IsolationForest

In [54]:
from hana_ml.algorithms.pal.preprocessing import IsolationForest
isof = IsolationForest(random_state=2, thread_ratio=0)

isof.fit(data=acdoca_hdf, key='ID', features=['Financial Account Type', 'Amount (USD)', 'Amount (Transaction)']) #issue with categorial feature in model: 'Financial Account Type'
#isof.fit(data=acdoca_hdf, key='ID', features=['Amount (USD)', 'Amount (Transaction)'])

#res=isof.fit_predict(data=acdoca_hdf, key='ID', features=['Amount (USD)', 'Amount (Transaction)'], contamination=0.25)                  
#res.collect()

In [55]:
print(myconn.last_execute_statement)

DO
BEGIN
DECLARE param_name VARCHAR(5000) ARRAY;
DECLARE int_value INTEGER ARRAY;
DECLARE double_value DOUBLE ARRAY;
DECLARE string_value VARCHAR(5000) ARRAY;
param_name[1] := N'SEED';
int_value[1] := 2;
double_value[1] := NULL;
string_value[1] := NULL;
param_name[2] := N'THREAD_RATIO';
int_value[2] := NULL;
double_value[2] := 0;
string_value[2] := NULL;
param_name[3] := N'HAS_ID';
int_value[3] := 1;
double_value[3] := NULL;
string_value[3] := NULL;
params = UNNEST(:param_name, :int_value, :double_value, :string_value);
in_0 = SELECT "ID", "Financial Account Type", "Amount (USD)", "Amount (Transaction)" FROM (SELECT * FROM (SELECT CAST(ROW_NUMBER() OVER() AS INTEGER) + 0 AS "ID", * FROM (SELECT * FROM "ACDOCA")) AS "DT_97" WHERE "Company Code" = 'CC01' AND "Profit Center"='PC002') AS "DT_98";
CALL _SYS_AFL.PAL_ISOLATION_FOREST(:in_0, :params, out_0);
CREATE LOCAL TEMPORARY COLUMN TABLE "#PAL_ISOLATION_FOREST_MODEL_TBL_6_3F2BCDFD_78EE_11F0_B066_F47B09A01846" AS (SELECT * FROM :out_0);
E

In [56]:
res = isof.predict(data=acdoca_hdf, key='ID', features=['Financial Account Type', 'Amount (USD)', 'Amount (Transaction)'],
#res = isof.predict(data=acdoca_hdf, key='ID', features=['Amount (USD)', 'Amount (Transaction)'],
                       contamination=0.25)
res.head(5).collect()

,ID,SCORE,LABEL
0,1,0.507132,1
1,3,0.541918,-1
2,4,0.543095,-1
3,6,0.479810,1
4,13,0.511375,1


In [57]:
res = isof.predict(data=acdoca_hdf, key='ID', features=['Financial Account Type', 'Amount (USD)', 'Amount (Transaction)'],
#res = isof.predict(data=acdoca_hdf, key='ID', features=['Amount (USD)', 'Amount (Transaction)'],
                       contamination=0.25,
                       show_explainer=True, explain_scope='outliers', top_k_attributions=5)
res.head(5).collect()


,ID,SCORE,LABEL,REASON_CODE
0,1,0.507132,1,None
1,3,0.541918,-1,"[{""attr"":""Financial Account Type"",""val"":-0.5844153276967007,""pct"":51.6},{""attr"":""Amount (USD)"",""val"":0.2823021538957196,""pct"":24.9},{""attr"":""Amount (Transaction)"",""val"":0.26574199253489885,""pct"":23.5}]"
2,4,0.543095,-1,"[{""attr"":""Amount (Transaction)"",""val"":0.6453605198892147,""pct"":49.5},{""attr"":""Financial Account Type"",""val"":0.44036125232480269,""pct"":33.8},{""attr"":""Amount (USD)"",""val"":-0.21695388528558003,""pct"":16.700000000000004}]"
3,6,0.479810,1,None
4,13,0.511375,1,None


In [58]:
# Review the predicted results
pd.set_option('max_colwidth', None)
res.select('ID', 'SCORE', 'LABEL', 'REASON_CODE' 
                ,('json_query("REASON_CODE", \'$[0].attr\')', 'Top1'), ('json_query("REASON_CODE", \'$[0].pct\')', 'PCT_1') 
                ,('json_query("REASON_CODE", \'$[1].attr\')', 'Top2'), ('json_query("REASON_CODE", \'$[1].pct\')', 'PCT_2') 
                ,('json_query("REASON_CODE", \'$[2].attr\')', 'Top3'), ('json_query("REASON_CODE", \'$[2].pct\')', 'PCT_3') 
          ).filter("LABEL < 0").head(30).collect() 

,ID,SCORE,LABEL,REASON_CODE,Top1,PCT_1,Top2,PCT_2,Top3,PCT_3
0,3,0.541918,-1,"[{""attr"":""Financial Account Type"",""val"":-0.5844153276967007,""pct"":51.6},{""attr"":""Amount (USD)"",""val"":0.2823021538957196,""pct"":24.9},{""attr"":""Amount (Transaction)"",""val"":0.26574199253489885,""pct"":23.5}]","""Financial Account Type""",51.6,"""Amount (USD)""",24.9,"""Amount (Transaction)""",23.5
1,4,0.543095,-1,"[{""attr"":""Amount (Transaction)"",""val"":0.6453605198892147,""pct"":49.5},{""attr"":""Financial Account Type"",""val"":0.44036125232480269,""pct"":33.8},{""attr"":""Amount (USD)"",""val"":-0.21695388528558003,""pct"":16.700000000000004}]","""Amount (Transaction)""",49.5,"""Financial Account Type""",33.8,"""Amount (USD)""",16.700000000000004
2,30,0.536671,-1,"[{""attr"":""Amount (Transaction)"",""val"":0.9778864918931063,""pct"":66.1},{""attr"":""Amount (USD)"",""val"":-0.3007482441873323,""pct"":20.3},{""attr"":""Financial Account Type"",""val"":-0.2006610056025669,""pct"":13.6}]","""Amount (Transaction)""",66.1,"""Amount (USD)""",20.3,"""Financial Account Type""",13.6
3,56,0.621685,-1,"[{""attr"":""Financial Account Type"",""val"":-1.0704668154950677,""pct"":85.5},{""attr"":""Amount (Transaction)"",""val"":-0.09937422676406832,""pct"":7.9},{""attr"":""Amount (USD)"",""val"":0.08307127870340622,""pct"":6.6}]","""Financial Account Type""",85.5,"""Amount (Transaction)""",7.9,"""Amount (USD)""",6.6
4,59,0.563833,-1,"[{""attr"":""Amount (Transaction)"",""val"":0.7954793436615294,""pct"":51.2},{""attr"":""Financial Account Type"",""val"":-0.46927638486571218,""pct"":30.200000000000004},{""attr"":""Amount (USD)"",""val"":-0.28972460618791359,""pct"":18.6}]","""Amount (Transaction)""",51.2,"""Financial Account Type""",30.200000000000004,"""Amount (USD)""",18.6
5,102,0.550273,-1,"[{""attr"":""Financial Account Type"",""val"":-0.9236730026632767,""pct"":68.6},{""attr"":""Amount (Transaction)"",""val"":0.38432518029852949,""pct"":28.5},{""attr"":""Amount (USD)"",""val"":0.03840604865369391,""pct"":2.9}]","""Financial Account Type""",68.6,"""Amount (Transaction)""",28.5,"""Amount (USD)""",2.9
6,114,0.534876,-1,"[{""attr"":""Financial Account Type"",""val"":-0.8461465520679107,""pct"":43.2},{""attr"":""Amount (USD)"",""val"":0.7611826641366812,""pct"":38.8},{""attr"":""Amount (Transaction)"",""val"":0.35372646767131879,""pct"":18.0}]","""Financial Account Type""",43.2,"""Amount (USD)""",38.8,"""Amount (Transaction)""",18.0
7,119,0.536026,-1,"[{""attr"":""Financial Account Type"",""val"":-1.0630298386444637,""pct"":45.0},{""attr"":""Amount (USD)"",""val"":0.9937022886557001,""pct"":42.1},{""attr"":""Amount (Transaction)"",""val"":0.30489772948768958,""pct"":12.9}]","""Financial Account Type""",45.0,"""Amount (USD)""",42.1,"""Amount (Transaction)""",12.9
8,121,0.599426,-1,"[{""attr"":""Financial Account Type"",""val"":-1.2034609235405476,""pct"":67.6},{""attr"":""Amount (Transaction)"",""val"":-0.4941185426899669,""pct"":27.7},{""attr"":""Amount (USD)"",""val"":0.08315631330603465,""pct"":4.699999999999999}]","""Financial Account Type""",67.6,"""Amount (Transaction)""",27.7,"""Amount (USD)""",4.699999999999999
9,143,0.582373,-1,"[{""attr"":""Amount (Transaction)"",""val"":0.70157674980096,""pct"":44.0},{""attr"":""Financial Account Type"",""val"":-0.45949970091078787,""pct"":28.8},{""attr"":""Amount (USD)"",""val"":-0.4332339817535791,""pct"":27.200000000000004}]","""Amount (Transaction)""",44.0,"""Financial Account Type""",28.8,"""Amount (USD)""",27.200000000000004


In [42]:
#join #.rename_columns({"B":"B2"})
acdoca_hdf.select('ID', 'Amount (USD)', 'Amount (Transaction)').set_index("ID").join(res.set_index("ID")).sort('SCORE', desc=True).collect()

,ID,Amount (USD),Amount (Transaction),SCORE,LABEL,REASON_CODE
0,496,19772.27,19772.27,0.706856,-1,"[{""attr"":""Amount (USD)"",""val"":-2.9235808472626299,""pct"":98.19999999999999},{""attr"":""Amount (Transaction)"",""val"":0.05505784736119764,""pct"":1.8}]"
1,143,37.13,-37.13,0.641794,-1,"[{""attr"":""Amount (Transaction)"",""val"":-2.2540583741547017,""pct"":58.0},{""attr"":""Amount (USD)"",""val"":-1.6298494934351507,""pct"":42.0}]"
2,56,19135.96,19135.96,0.638262,-1,"[{""attr"":""Amount (USD)"",""val"":-2.6470002830027097,""pct"":85.69999999999999},{""attr"":""Amount (Transaction)"",""val"":0.44285833182641057,""pct"":14.3}]"
3,179,18116.34,18116.34,0.611059,-1,"[{""attr"":""Amount (USD)"",""val"":-2.585932668552592,""pct"":87.5},{""attr"":""Amount (Transaction)"",""val"":0.3705882236486529,""pct"":12.5}]"
4,378,16896.19,16896.19,0.609536,-1,"[{""attr"":""Amount (USD)"",""val"":-2.215150761484544,""pct"":91.89999999999999},{""attr"":""Amount (Transaction)"",""val"":0.19584888089293743,""pct"":8.1}]"
...,...,...,...,...,...,...
162,61,4116.27,-4116.27,0.459214,1,None
163,408,7824.09,-7824.09,0.458312,1,None
164,442,10186.79,10186.79,0.456669,1,None
165,1,4158.39,-4158.39,0.455681,1,None


## Section 1.3 - Model storage and retrieval of outlier IF models

## Section 1.4 - Outlier in subgroups: Massive Isolation Forest

In [19]:
mif = IsolationForest(massive=True, random_state=2, group_params={'Group_1': {'n_estimators':50}})

In [20]:
mif = IsolationForest(massive=True, random_state=2
                      #, group_params={'Group_1': {'n_estimators':50}
                     )
mif.fit(data=acdoca_hdf, key="ID", group_key="G/L Account", features=['Amount (USD)', 'Amount (Transaction)'])

res, err = mif.predict(data=acdoca_hdf, key="ID", group_key="G/L Account", features=['Amount (USD)', 'Amount (Transaction)']
                       #,group_params={'Group_1': {'contamination':0.2}
                      )


In [21]:
display(res.sort('SCORE', desc=True).head(10).collect())
#display(err.head(10).collect())

,GROUP_ID,ID,SCORE,LABEL
0,720000,292,0.696500,-1
1,630000,305,0.696500,-1
2,580000,291,0.652863,-1
3,480000,135,0.652863,-1
4,520000,444,0.652863,-1
5,510000,3,0.643945,-1
6,700000,484,0.639670,-1
7,530000,220,0.621176,-1
8,550000,161,0.619574,-1
9,450000,121,0.608464,-1


In [22]:
# join original value with outlier data
acdoca_hdf.select('ID', 'G/L Account', 'Amount (USD)', 'Amount (Transaction)').set_index("ID").join(res.set_index("ID")).sort('SCORE', desc=True).collect()

,ID,G/L Account,Amount (USD),Amount (Transaction),GROUP_ID,SCORE,LABEL
0,292,720000,6597.85,-6597.85,720000,0.696500,-1
1,305,630000,19376.51,-19376.51,630000,0.696500,-1
2,291,580000,18780.69,-18780.69,580000,0.652863,-1
3,135,480000,18774.22,18774.22,480000,0.652863,-1
4,444,520000,12358.72,-12358.72,520000,0.652863,-1
...,...,...,...,...,...,...,...
162,268,760000,1282.54,-1282.54,760000,0.317216,1
163,137,790000,5082.50,-5082.50,790000,0.317216,1
164,196,750000,14606.50,-14606.50,750000,0.317216,1
165,179,460000,18116.34,18116.34,460000,0.317216,1


In [44]:
# Massive with reason code
res, err = mif.predict(data=acdoca_hdf, key="ID", group_key="G/L Account", features=['Amount (USD)', 'Amount (Transaction)']
                       #,group_params={'Group_1': {'contamination':0.2}
                       ,show_explainer=True, explain_scope='outliers', top_k_attributions=5
                      )
display(res.sort('SCORE', desc=True).head(10).collect())

,GROUP_ID,ID,SCORE,LABEL,REASON_CODE
0,630000,305,0.696500,-1,"[{""attr"":""Amount (USD)"",""val"":-0.520014144920211,""pct"":99.3},{""attr"":""Amount (Transaction)"",""val"":0.003813871880006585,""pct"":0.7}]"
1,720000,292,0.696500,-1,"[{""attr"":""Amount (USD)"",""val"":0.0,""pct"":50.0},{""attr"":""Amount (Transaction)"",""val"":0.0,""pct"":50.0}]"
2,520000,444,0.652863,-1,"[{""attr"":""Amount (USD)"",""val"":0.0,""pct"":50.0},{""attr"":""Amount (Transaction)"",""val"":0.0,""pct"":50.0}]"
3,480000,135,0.652863,-1,"[{""attr"":""Amount (USD)"",""val"":-0.4498360879241077,""pct"":70.9},{""attr"":""Amount (Transaction)"",""val"":0.18431571292275687,""pct"":29.1}]"
4,580000,291,0.652863,-1,"[{""attr"":""Amount (USD)"",""val"":0.0,""pct"":50.0},{""attr"":""Amount (Transaction)"",""val"":0.0,""pct"":50.0}]"
5,510000,3,0.643945,-1,"[{""attr"":""Amount (USD)"",""val"":-0.3255878400824326,""pct"":95.89999999999999},{""attr"":""Amount (Transaction)"",""val"":-0.014062094166477503,""pct"":4.1}]"
6,700000,484,0.639670,-1,"[{""attr"":""Amount (Transaction)"",""val"":0.2530000000000003,""pct"":62.0},{""attr"":""Amount (USD)"",""val"":-0.15499999999999987,""pct"":38.0}]"
7,530000,220,0.621176,-1,"[{""attr"":""Amount (USD)"",""val"":-0.3449482838104983,""pct"":75.19999999999999},{""attr"":""Amount (Transaction)"",""val"":0.11383880825506998,""pct"":24.8}]"
8,550000,161,0.619574,-1,"[{""attr"":""Amount (USD)"",""val"":-0.2801066595973681,""pct"":57.7},{""attr"":""Amount (Transaction)"",""val"":0.20500125306054418,""pct"":42.300000000000007}]"
9,450000,121,0.608464,-1,"[{""attr"":""Amount (USD)"",""val"":-0.15448870829103865,""pct"":95.39999999999999},{""attr"":""Amount (Transaction)"",""val"":0.0075188195149355559,""pct"":4.6}]"


# Appendix

## Outlier data

In [69]:
import random
import pandas as pd
SourceLedger = ['SL01', 'SL02', 'SL03','SL04','SL05','SL06','SL07',]
CompanyCode = ['CC01', 'CC02','CC03','CC04','CC05','CC06','CC07','CC08','CC09']
GLAccount = [str(x) for x in range(400000, 800000, 10000)]
profit_centers = [str(x) for x in range(10, 99, 1)]
cost_centers = [str(x) for x in range(10, 99, 1)]
FiscalYear = ['2024']
FiscalPeriod = ['01']
FiscalPeriodDay = [str(x) for x in range(1, 90, 1)]
AccountingDocument = [str(x) for x in range(100000000, 999000000, 10000)]
GLLineItem = [str(x) for x in range(100000, 999000, 10000)]
segments = ['S1', 'S2', 'S3','S4', 'S5']
GlobalCurrency = ['USD']
dc_indicators = ['S', 'H']           #'Debit/Credit'
doc_types = ['SA', 'SB', 'SC', 'SD'] #'Accounting Document Type'
# AB General document, DG Customer credit memo, DZ Customer payment, DR Customer invoice, KZ Vendor payment, KG Vendor credit memo, KN Vendor net invoice and credit memo
# KR Vendor invoice, SA General G/L accounts
# FP Financial Processing, https://accounting.uci.edu/support/fiscal-officers/coa/document-types.php
tx_types = ['TA01', 'TA02', 'TA03']  #TransactionType
fin_types = ['P+L Statement', 'Balance Sheet Asset', 'Balance Sheet Liability', 'Equity'] #'Financial Account Type'
data = []
for _ in range(500000):
   amount = round(random.uniform(-20000, 20000), 2)
   data.append([
       random.choice(SourceLedger),
       random.choice(CompanyCode),
       random.choice(GLAccount),
       random.choice(profit_centers),
       random.choice(cost_centers),
       random.choice(FiscalYear),
       random.choice(FiscalPeriod),
       random.choice(FiscalPeriodDay),
       random.choice(AccountingDocument),
       random.choice(GLLineItem),
       random.choice(segments),
       random.choice(doc_types),
       random.choice(tx_types),
       random.choice(fin_types),
       random.choice(dc_indicators),
       random.choice(GlobalCurrency),
       abs(amount),
       amount
   ])
df = pd.DataFrame(data, columns=[
    'SourceLedger', 'CompanyCode', 'GLAccount', 'ProfitCenter', 'CostCenter',
    'FiscalYear', 'FiscalPeriod', 'FiscalPeriodDay',
    'AccountingDocument', 'GLLineItem', 'Segment', 
    'AccountingDocumentType', 'TransactionType', 'FinancialAccountType',
    'DebitCredit','GlobalCurrency','AmountGC', 'AmountT'
])
df.to_csv('acdoca2_data.csv', index=False)

In [70]:
from hana_ml.dataframe import create_dataframe_from_pandas
import pandas as pd
acdoca_hdf = dataframe.create_dataframe_from_pandas(
        myconn,
        df,
        table_name="ACDOCA2",
        force=True,
        replace=True,
        drop_exist_tab=True
        #,table_structure={"Pregnancies": "INT", "Glucose" : "INT", "BloodPressure" : "INT", "SkinThickness" : "INT", "Insulin": "INT", "BMI"  : "DOUBLE", "DiabetesPedigreeFunction"  : "DOUBLE", "Age": "INT", "CLASS": "NVARCHAR(10)"}
        )
print(acdoca_hdf.select_statement)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:38<00:00,  3.49s/it]

SELECT "SourceLedger", "CompanyCode", "GLAccount", "ProfitCenter", "CostCenter", "FiscalYear", "FiscalPeriod", "FiscalPeriodDay", "AccountingDocument", "GLLineItem", "Segment", "AccountingDocumentType", "TransactionType", "FinancialAccountType", "DebitCredit", "GlobalCurrency", COALESCE("AmountGC", 0) AS "AmountGC", COALESCE("AmountT", 0) AS "AmountT" FROM (SELECT * FROM "ACDOCA2") dt


In [90]:
acdoca_hdf=myconn.table('ACDOCA2')

In [91]:
acdoca_hdf.head(2).collect()

,SourceLedger,CompanyCode,GLAccount,ProfitCenter,CostCenter,FiscalYear,FiscalPeriod,FiscalPeriodDay,AccountingDocument,GLLineItem,Segment,AccountingDocumentType,TransactionType,FinancialAccountType,DebitCredit,GlobalCurrency,AmountGC,AmountT
0,SL04,CC01,770000,93,27,2024,01,62,471370000,100000,S2,SC,TA02,Balance Sheet Liability,H,USD,10719.17,-10719.17
1,SL05,CC07,620000,78,50,2024,01,67,918070000,230000,S2,SC,TA02,Balance Sheet Liability,H,USD,18102.66,-18102.66


In [92]:
acdoca_hdf=acdoca_hdf.select('*', ('ADD_DAYS(TO_DATE(\'2024-01-01\', \'YYYY-MM-DD\'), "FiscalPeriodDay")','DateKey')).to_head('DateKey').drop('FiscalPeriodDay')
acdoca_hdf.head(2).collect()
#acdoca_hdf.select('*', ('ADD_DAYS(TO_DATE(\'2024-01-01\', \'YYYY-MM-DD\'), "FiscalPeriodDay") as DateKey')).head(2).collect()

,DateKey,SourceLedger,CompanyCode,GLAccount,ProfitCenter,CostCenter,FiscalYear,FiscalPeriod,AccountingDocument,GLLineItem,Segment,AccountingDocumentType,TransactionType,FinancialAccountType,DebitCredit,GlobalCurrency,AmountGC,AmountT
0,2024-03-03,SL04,CC01,770000,93,27,2024,01,471370000,100000,S2,SC,TA02,Balance Sheet Liability,H,USD,10719.17,-10719.17
1,2024-03-08,SL05,CC07,620000,78,50,2024,01,918070000,230000,S2,SC,TA02,Balance Sheet Liability,H,USD,18102.66,-18102.66


In [94]:
#Test expressions
acdoca_hdf.sort('DateKey', desc=True).head(2).collect()

,DateKey,SourceLedger,CompanyCode,GLAccount,ProfitCenter,CostCenter,FiscalYear,FiscalPeriod,AccountingDocument,GLLineItem,Segment,AccountingDocumentType,TransactionType,FinancialAccountType,DebitCredit,GlobalCurrency,AmountGC,AmountT
0,2024-03-30,SL04,CC03,700000,13,73,2024,01,912490000,170000,S2,SC,TA02,Balance Sheet Liability,H,USD,9243.26,-9243.26
1,2024-03-30,SL01,CC02,490000,17,76,2024,01,742580000,490000,S2,SC,TA02,Balance Sheet Liability,H,USD,15150.16,15150.16


In [95]:
# Feature Enrichment to the DF
def feature_engineering(hana_df):
        """
         This function perform feature enrichment to the dataframe to prepare for anomaly detection.
          """
        hana_df = hana_df.sort('DateKey', desc=True)
        hana_df = hana_df.select('*',  ('CASE WHEN "AmountGC" <> 0 THEN '  'SIGN("AmountGC") * LN(ABS("AmountGC" + 1e-6)) '  'ELSE 0 END',  'AmountGC_lognorm'))
        key_cols = ['SourceLedger', 'CompanyCode', 'GLAccount', 'ProfitCenter']
        orig_key = '_'.join(str(x) for x in key_cols)
        hana_df = hana_df.concat_columns(columns=key_cols, separator="_")
        hana_df = hana_df.rename_columns({orig_key: 'PK'})
        #hana_df = hana_df.rename_columns({'x_serial_key': 'primary_date'})
        balance_sum_df = hana_df.agg([('sum', 'AmountGC', 'AmountGC_sum')], group_by=['PK', 'DateKey', 'SourceLedger', 'FiscalYear', 'CompanyCode', 'GlobalCurrency']) #?"FiscalPeriod"
        print(balance_sum_df.shape) 
        balance_sum_df = balance_sum_df.select('*', ('AVG("AmountGC_sum") OVER (PARTITION BY "PK" ORDER BY "DateKey" ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)', 'AmountGC_sum_mean'))
        balance_sum_df = balance_sum_df.select('*', ('STDDEV("AmountGC_sum") OVER (PARTITION BY "PK" ORDER BY "DateKey" ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)', 'AmountGC_sum_stddev'))
        balance_sum_df = balance_sum_df.select('*', ('CASE WHEN "AmountGC_sum_stddev" = 0 THEN 0 ELSE (("AmountGC_sum" - "AmountGC_sum_mean") / "AmountGC_sum_stddev") END', 'AmountGC_zscore'))
        balance_sum_df = balance_sum_df.select('*', ('CASE WHEN "AmountGC_sum" <> 0 THEN '  'SIGN("AmountGC_sum") * LN(ABS("AmountGC_sum" + 1e-6)) '  'ELSE 0 END',  'AmountGC_sum_lognorm' ))  
        balance_sum_df = balance_sum_df.select('*', ('LAG("AmountGC_sum", 1) OVER (PARTITION BY "PK" ORDER BY "DateKey")', 'AmountGC_sum_lag_tmp'))
        balance_sum_df = balance_sum_df.select('*', ('COALESCE("AmountGC_sum_lag_tmp", 0)', 'AmountGC_sum_lag'))
        balance_sum_df = balance_sum_df.drop(["AmountGC_sum_lag_tmp"])  
        balance_sum_df = balance_sum_df.select('*', ('CASE WHEN "AmountGC_sum_lag" <> 0 THEN ' 'SIGN("AmountGC_sum_lag") * LN(ABS("AmountGC_sum_lag" + 1e-6)) '  'ELSE 0 END', 'AmountGC_sum_lag_lognorm'))
        balance_sum_df = balance_sum_df.select('*', ('"AmountGC_sum" - "AmountGC_sum_lag"', "AmountGC_sum_diff"))
        balance_sum_df = balance_sum_df.select('*', ('CASE WHEN "AmountGC_sum_diff" <> 0 THEN ' 'SIGN("AmountGC_sum_diff") * LN(ABS("AmountGC_sum_diff" + 1e-6)) '  'ELSE 0 END',  'AmountGC_sum_diff_lognorm'))

        cat_cols = ['SourceLedger', 'FiscalYear', 'CompanyCode', 'GlobalCurrency']
        #balance_sum_df = Preprocessing(name="OneHotEncoder").fit_transform(data=balance_sum_df, features=cat_cols)
        
        hana_df_enriched = hana_df.set_index(['PK', 'DateKey']).join(balance_sum_df.set_index(['PK', 'DateKey']))
        
        return hana_df_enriched, balance_sum_df

In [97]:
acdoca_hdf.columns

['DateKey',
 'SourceLedger',
 'CompanyCode',
 'GLAccount',
 'ProfitCenter',
 'CostCenter',
 'FiscalYear',
 'FiscalPeriod',
 'AccountingDocument',
 'GLLineItem',
 'Segment',
 'AccountingDocumentType',
 'TransactionType',
 'FinancialAccountType',
 'DebitCredit',
 'GlobalCurrency',
 'AmountGC',
 'AmountT']

In [98]:
# perform the feature engineering
enriched_df, balance_sum_df = feature_engineering(acdoca_hdf)

[493758, 7]


In [103]:
 balance_sum_df.sort(['PK', 'DateKey']).head(5).collect()

,PK,DateKey,SourceLedger,FiscalYear,CompanyCode,GlobalCurrency,AmountGC_sum,AmountGC_sum_mean,AmountGC_sum_stddev,AmountGC_zscore,AmountGC_sum_lognorm,AmountGC_sum_lag,AmountGC_sum_lag_lognorm,AmountGC_sum_diff,AmountGC_sum_diff_lognorm
0,SL01_CC01_400000_10,2024-01-12,SL01,2024,CC01,USD,1135.04,1135.040000,0.000000,0.000000,7.034423,0.00,0.000000,1135.04,7.034423
1,SL01_CC01_400000_11,2024-01-03,SL01,2024,CC01,USD,11358.40,11358.400000,0.000000,0.000000,9.337713,0.00,0.000000,11358.40,9.337713
2,SL01_CC01_400000_11,2024-02-25,SL01,2024,CC01,USD,6588.72,8973.560000,3372.673072,-0.707107,8.793114,11358.40,9.337713,-4769.68,-8.470034
3,SL01_CC01_400000_11,2024-03-01,SL01,2024,CC01,USD,6801.66,8249.593333,2694.409959,-0.537384,8.824922,6588.72,8.793114,212.94,5.361010
4,SL01_CC01_400000_11,2024-03-05,SL01,2024,CC01,USD,17633.34,10595.530000,5182.043232,1.358115,9.777547,6801.66,8.824922,10831.68,9.290230


In [102]:
# view the head of the enriched table
enriched_df.sort(['PK', 'DateKey']).head(5).collect()

,DateKey,SourceLedger,CompanyCode,GLAccount,ProfitCenter,CostCenter,FiscalYear,FiscalPeriod,AccountingDocument,GLLineItem,...,GlobalCurrency,AmountGC_sum,AmountGC_sum_mean,AmountGC_sum_stddev,AmountGC_zscore,AmountGC_sum_lognorm,AmountGC_sum_lag,AmountGC_sum_lag_lognorm,AmountGC_sum_diff,AmountGC_sum_diff_lognorm
0,2024-01-12,SL01,CC01,400000,10,28,2024,01,237000000,150000,...,USD,1135.04,1135.040000,0.000000,0.000000,7.034423,0.00,0.000000,1135.04,7.034423
1,2024-01-03,SL01,CC01,400000,11,96,2024,01,624510000,220000,...,USD,11358.40,11358.400000,0.000000,0.000000,9.337713,0.00,0.000000,11358.40,9.337713
2,2024-02-25,SL01,CC01,400000,11,67,2024,01,452530000,530000,...,USD,6588.72,8973.560000,3372.673072,-0.707107,8.793114,11358.40,9.337713,-4769.68,-8.470034
3,2024-03-01,SL01,CC01,400000,11,89,2024,01,800310000,910000,...,USD,6801.66,8249.593333,2694.409959,-0.537384,8.824922,6588.72,8.793114,212.94,5.361010
4,2024-03-05,SL01,CC01,400000,11,71,2024,01,609610000,770000,...,USD,17633.34,10595.530000,5182.043232,1.358115,9.777547,6801.66,8.824922,10831.68,9.290230
5,2024-03-16,SL01,CC01,400000,13,10,2024,01,742210000,250000,...,USD,5463.90,5463.900000,0.000000,0.000000,8.605918,0.00,0.000000,5463.90,8.605918
6,2024-01-27,SL01,CC01,400000,14,17,2024,01,656440000,300000,...,USD,8807.99,8807.990000,0.000000,0.000000,9.083415,0.00,0.000000,8807.99,9.083415
7,2024-03-12,SL01,CC01,400000,14,82,2024,01,592540000,520000,...,USD,11853.03,10330.510000,2153.168433,0.707107,9.380339,8807.99,9.083415,3045.04,8.021269
8,2024-02-10,SL01,CC01,400000,15,73,2024,01,979990000,910000,...,USD,12296.97,12296.970000,0.000000,0.000000,9.417108,0.00,0.000000,12296.97,9.417108
9,2024-03-03,SL01,CC01,400000,15,91,2024,01,620710000,220000,...,USD,14568.64,13432.805000,1606.313262,0.707107,9.586627,12296.97,9.417108,2271.67,7.728271


In [ ]:
import random
import pandas as pd
company_codes = ['CC01']
gl_accounts = [str(x) for x in range(400000, 800000, 10000)]
profit_centers = ['PC001', 'PC002', 'PC003']
cost_centers = ['C101', 'C102', 'C103']
functional_areas = ['FA01', 'FA02']
business_areas = ['BA01', 'BA02', 'BA03']
segments = ['S1', 'S2', 'S3']
dc_indicators = ['S', 'H']
doc_types = ['SA', 'SB', 'SC', 'SD']
tx_types = ['TA01', 'TA02', 'TA03']
fin_types = ['P+L Statement', 'Balance Sheet Asset', 'Balance Sheet Liability', 'Equity']
data = []
for _ in range(500):
   amount = round(random.uniform(-20000, 20000), 2)
   data.append([
       random.choice(company_codes),
       random.choice(gl_accounts),
       random.choice(profit_centers),
       random.choice(cost_centers),
       random.choice(functional_areas),
       random.choice(business_areas),
       random.choice(segments),
       random.choice(dc_indicators),
       random.choice(doc_types),
       random.choice(tx_types),
       random.choice(fin_types),
       abs(amount),
       amount
   ])
df = pd.DataFrame(data, columns=[
   'Company Code', 'G/L Account', 'Profit Center', 'Cost Center',
   'Functional Area', 'Business Area', 'Segment', 'Debit/Credit',
   'Accounting Document Type', 'Transaction Type', 'Financial Account Type',
   'Amount (USD)', 'Amount (Transaction)'
])
df.to_csv('acdoca_data.csv', index=False)

In [5]:
from hana_ml.dataframe import create_dataframe_from_pandas
import pandas as pd
acdoca_hdf = dataframe.create_dataframe_from_pandas(
        myconn,
        df,
        table_name="ACDOCA",
        force=True,
        replace=True,
        drop_exist_tab=True
        #,table_structure={"Pregnancies": "INT", "Glucose" : "INT", "BloodPressure" : "INT", "SkinThickness" : "INT", "Insulin": "INT", "BMI"  : "DOUBLE", "DiabetesPedigreeFunction"  : "DOUBLE", "Age": "INT", "CLASS": "NVARCHAR(10)"}
        )
print(acdoca_hdf.select_statement)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.56it/s]

SELECT "Company Code", "G/L Account", "Profit Center", "Cost Center", "Functional Area", "Business Area", "Segment", "Debit/Credit", "Accounting Document Type", "Transaction Type", "Financial Account Type", COALESCE("Amount (USD)", 0) AS "Amount (USD)", COALESCE("Amount (Transaction)", 0) AS "Amount (Transaction)" FROM (SELECT * FROM "ACDOCA") dt


In [59]:
display(acdoca_hdf.collect())

,ID,Company Code,G/L Account,Profit Center,Cost Center,Functional Area,Business Area,Segment,Debit/Credit,Accounting Document Type,Transaction Type,Financial Account Type,Amount (USD),Amount (Transaction)
0,1,CC01,550000,PC002,C101,FA01,BA02,S1,S,SC,TA02,P+L Statement,4158.39,-4158.39
1,3,CC01,510000,PC002,C102,FA01,BA03,S1,S,SD,TA03,P+L Statement,18333.69,-18333.69
2,4,CC01,470000,PC002,C102,FA01,BA02,S2,H,SC,TA01,Equity,1825.55,-1825.55
3,6,CC01,410000,PC002,C101,FA02,BA01,S3,S,SC,TA01,Balance Sheet Liability,10073.36,-10073.36
4,13,CC01,640000,PC002,C103,FA01,BA01,S1,S,SB,TA03,Balance Sheet Asset,12119.38,12119.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,484,CC01,700000,PC002,C103,FA01,BA02,S3,S,SB,TA02,Balance Sheet Asset,12661.97,12661.97
163,485,CC01,530000,PC002,C101,FA02,BA01,S3,H,SA,TA02,Balance Sheet Asset,7931.05,-7931.05
164,489,CC01,480000,PC002,C102,FA02,BA02,S2,S,SC,TA02,Balance Sheet Asset,5533.86,5533.86
165,491,CC01,420000,PC002,C101,FA02,BA02,S1,H,SB,TA02,Equity,10298.93,10298.93


In [67]:
#testhdf=myconn.sql("Select count(*) as N, GLA from (Select distinct \"G/L Account\" as GLA, \"Financial Account Type\" FROM ACDOCA) group by GLA order by 1 desc")
# 1 account hat 4 Account-Types, das geht nicht 
# 1 account entweder profit oder costcenter
# 1 profit center ist Fest einer FuncArea/BusArea/Segment zugewiese
# 1 account Entweder Debit oder Credit
# ? Accounting Doc Type ?
# ? Transaction Type ?
display(testhdf.collect())

,N,GLA
0,4,400000
1,4,780000
2,4,770000
3,4,760000
4,4,750000
5,4,740000
6,4,730000
7,4,700000
8,4,690000
9,4,670000
